# Problema2 - REALIZAR SOBRE ARCHIVO .PY  (NO JUPYTER) 


Lea el archivo 'winemag-data-130k-v2.csv' y realice lo siguiente:

- Explore el dataframe según lo visto en clase
- Realice al menos  4 renombre de columna y 3 creación de nuevas columnas según criterio. Deberá crear las columnas que crea conveniente. 
    Ejemplo: Según el país etiquetelos según continente.

- Genere 4 reportes distintos(podria agrupando, filtrar, ordenar, etc). Deberá elegir que reportes realizar
    - Ejemplo: Por contienente mostrar los vinos mejor puntuados
    - Ejemplo2: Promedio de precio de vino y cantidad de reviews obtenidos según pais. Ordenado de mejor a peor.



- Exporte los 4 reportes a 4 formatos de archivos diversos, csv, excel, sqllite, mongodb...
   
   Para guardar datos de datos agrupados, puede revisar esta solución https://stackoverflow.com/questions/25789264/pandas-dataframegroupby-export-to-excel

- Envie uno de los reportes generados por correo, mostrar un pantallazo del correo enviado




### Datos 

winemag-data-130k-v2.csv
- https://github.com/gdelgador/ProgramacionPython202401/blob/main/Modulo5/src/winemag-data-130k-v2.csv

País según continente:
- https://gist.githubusercontent.com/kintero/7d1db891401f56256c79/raw/a61f6d0dda82c3f04d2e6e76c3870552ef6cf0c6/paises.csv
 






In [12]:
import pandas as pd

# Leer el archivo CSV
file_path = '/workspaces/PC5/data/winemag-data-130k-v2.csv'
df = pd.read_csv(file_path)

# Mostrar las primeras filas y nombres de columnas
print(df.head())
print(df.columns)


   Unnamed: 0   country                                        description  \
0           0     Italy  Aromas include tropical fruit, broom, brimston...   
1           1  Portugal  This is ripe and fruity, a wine that is smooth...   
2           2        US  Tart and snappy, the flavors of lime flesh and...   
3           3        US  Pineapple rind, lemon pith and orange blossom ...   
4           4        US  Much like the regular bottling from 2012, this...   

                          designation  points  price           province  \
0                        Vulkà Bianco      87    NaN  Sicily & Sardinia   
1                            Avidagos      87   15.0              Douro   
2                                 NaN      87   14.0             Oregon   
3                Reserve Late Harvest      87   13.0           Michigan   
4  Vintner's Reserve Wild Child Block      87   65.0             Oregon   

              region_1           region_2         taster_name  \
0              

In [14]:
# Renombrar columnas
df.rename(columns={
    'country': 'Country',
    'description': 'Description',
    'designation': 'Designation',
    'points': 'Points',
    'price': 'Price',
    'province': 'Province',
    'region_1': 'Region_1',
    'region_2': 'Region_2',
    'taster_name': 'Taster_Name',
    'taster_twitter_handle': 'Taster_Twitter_Handle',
    'title': 'Title',
    'variety': 'Variety',
    'winery': 'Winery'
}, inplace=True)

# Crear nuevas columnas
df['Continent'] = df['Country'].map({
    'Italy': 'Europe',
    'Portugal': 'Europe',
    'US': 'North America',
    'Spain': 'Europe',
    'France': 'Europe',
    'Germany': 'Europe',
    'Argentina': 'South America'
    # Añadir más países según sea necesario
})

df['Price_Category'] = pd.cut(df['Price'].fillna(0), bins=[0, 20, 50, 100, 200, float('inf')],
                              labels=['Cheap', 'Moderate', 'Expensive', 'Very Expensive', 'Luxury'])

df['Year'] = pd.to_datetime(df['Title'].str.extract(r'\((\d{4})\)', expand=False), format='%Y').dt.year

df['Is_Highly_Rated'] = df['Points'] >= 90


In [21]:
top_vins_by_continent = df.groupby('Continent').apply(lambda x: x.loc[x['Points'].idxmax()]).reset_index(drop=True)

avg_price_reviews = df.groupby('Country').agg(
    Average_Price=('Price', 'mean'),
    Reviews_Count=('Title', 'count')
).sort_values(by='Average_Price', ascending=False).reset_index()

price_distribution = df['Price_Category'].value_counts().reset_index()
price_distribution.columns = ['Price_Category', 'Count']

highly_rated_by_region = df[df['Is_Highly_Rated']].groupby('Region_1').agg(
    High_Rated_Count=('Title', 'count'),
    Average_Points=('Points', 'mean')
).reset_index()


/tmp/ipykernel_29299/3198175943.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_vins_by_continent = df.groupby('Continent').apply(lambda x: x.loc[x['Points'].idxmax()]).reset_index(drop=True)


In [ ]:
import sqlite3
import pymongo
from pymongo import MongoClient

# Exportar a CSV
top_vins_by_continent.to_csv('top_vins_by_continent.csv', index=False)
avg_price_reviews.to_csv('avg_price_reviews.csv', index=False)
price_distribution.to_csv('price_distribution.csv', index=False)
highly_rated_by_region.to_csv('highly_rated_by_region.csv', index=False)

# Exportar a Excel
with pd.ExcelWriter('reports.xlsx') as writer:
    top_vins_by_continent.to_excel(writer, sheet_name='Top Vins by Continent', index=False)
    avg_price_reviews.to_excel(writer, sheet_name='Avg Price Reviews', index=False)
    price_distribution.to_excel(writer, sheet_name='Price Distribution', index=False)
    highly_rated_by_region.to_excel(writer, sheet_name='Highly Rated by Region', index=False)

# Exportar a SQLite
conn = sqlite3.connect('reports.db')
top_vins_by_continent.to_sql('top_vins_by_continent', conn, if_exists='replace', index=False)
avg_price_reviews.to_sql('avg_price_reviews', conn, if_exists='replace', index=False)
price_distribution.to_sql('price_distribution', conn, if_exists='replace', index=False)
highly_rated_by_region.to_sql('highly_rated_by_region', conn, if_exists='replace', index=False)
conn.close()

# Exportar a MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['wine_reports']
db['top_vins_by_continent'].insert_many(top_vins_by_continent.to_dict('records'))
db['avg_price_reviews'].insert_many(avg_price_reviews.to_dict('records'))
db['price_distribution'].insert_many(price_distribution.to_dict('records'))
db['highly_rated_by_region'].insert_many(highly_rated_by_region.to_dict('records'))


In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from email.mime.text import MIMEText

# Configuración del correo
sender_email = 'email'
receiver_email = 'renato272000@gmail.com'
subject = 'Reporte de Vinos'
body = 'Adjunto el reporte solicitado.'

# Crear el mensaje
msg = MIMEMultipart()
msg['From'] = sender_email
msg['To'] = receiver_email
msg['Subject'] = subject
msg.attach(MIMEText(body, 'plain'))

# Adjuntar el archivo
filename = 'top_vins_by_continent.csv'
attachment = open(filename, 'rb')
part = MIMEBase('application', 'octet-stream')
part.set_payload(attachment.read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', f'attachment; filename={filename}')
msg.attach(part)

# Enviar el correo
with smtplib.SMTP('smtp.gmail.com', 587) as server:
    server.starttls()
    server.login(sender_email, 'your_password')  # Asegúrate de usar una contraseña segura
    server.sendmail(sender_email, receiver_email, msg.as_string())

print("Correo enviado.")
